In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


import os
import sys
import copy
from torchsummary import summary


# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/

#device config
torch.cuda.set_device(1)#Select GPU device 1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Get Packages
sys.path.append('./Tools')

#Network architectures
from ResNet18_CIFAR10 import ResNet18_CIFAR10_Model
from ResNet18_TINYIMAGENET import ResNet18_TINYIMAGENET_Model
from VGG16_CIFAR10 import VGG16_CIFAR10_Model
from VGG16_TINYIMAGENET import VGG16_TINYIMAGENET_Model

#Non Corrupted DataLoaders
from CIFAR10_LOADER import CIFAR10DataLoader
from TINYIMAGENET_LOADER import TINYIMAGENETDataLoader

#Non Corrupted DataLoaders
from CIFAR10C_LOADER import CIFAR10C_DataLoader #Each Corruption is a entry in the dictionary
from TINYIMAGENET_C_LOADER import TinyImagenetC_DataLoader #Each Corruption is a entry in the dictionary

#Error Metrics
from ErrorMetrics import Evaluate_Model_TOP1
from GeneralPruning import iterative_pruning


# Unpruned Models

In [2]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0.pt")
ResNet18_TINYIMAGENET_Weights=torch.load("./Networks/ResNet18_TINY_IMAGENET_V0.pt")
VGG16_CIFAR10_Weights=torch.load("./Networks/VGG16_CIFAR10_V0.pt")
VGG16_TINYIMAGENET_Weights=torch.load("./Networks/VGG16_TINY_IMAGENET_V0.pt")

#Unpruned Models
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

VGG16_CIFAR10_0P=VGG16_CIFAR10_Model()
VGG16_CIFAR10_0P.load_state_dict(VGG16_CIFAR10_Weights)

VGG16_TINYIMAGENET_0P=VGG16_TINYIMAGENET_Model()
VGG16_TINYIMAGENET_0P.load_state_dict(VGG16_TINYIMAGENET_Weights)


<All keys matched successfully>

# Load Datasets

In [3]:
from unittest import loader

Times=5
trainloader_CIFAR10, testloader_CIFAR10=CIFAR10DataLoader(batch_size=512*Times) #Load Data for CIFAR10
loader_CIFAR10C=CIFAR10C_DataLoader(batch_size=512*Times) #Load Data for CIFAR10C

Times=1
trainloader_TINYIMAGENET, testloader_TINYIMAGENET=TINYIMAGENETDataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENET
loader_TinyImagenetC=TinyImagenetC_DataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENETC

Files already downloaded and verified
Files already downloaded and verified


# CIFAR 10

## Check Effect of Pruning Rate

In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_05PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)

In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_025PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_01PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.05
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR_001PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.05 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#VGG16
model=copy.deepcopy(VGG16_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_CIFAR_05PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
 \nOriginal Model: VGG16 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model, device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)

In [ ]:
#VGG16
model=copy.deepcopy(VGG16_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_CIFAR_025PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
 \nOriginal Model: VGG16 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model, device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)

In [ ]:
#VGG16
model=copy.deepcopy(VGG16_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_CIFAR_01PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
 \nOriginal Model: VGG16 Trained with CIFAR10 Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)

# IMAGENET

In [ ]:
#Choose a corruption level because simulation every level is too much
TinyImagenetC_CorruptionClasses=list(loader_TinyImagenetC.keys())
TinyImagenetC_CorruptionLevels=list(loader_TinyImagenetC[TinyImagenetC_CorruptionClasses[0]].keys()) #['1', '2', '3', '4', '5']

#Choose a level
clevel='1'
TiniyImageNetCLevel_Loader={}#Filtered for a specifig level of corruption
for cname in loader_TinyImagenetC.keys():
    TiniyImageNetCLevel_Loader[cname]=loader_TinyImagenetC[cname][clevel]

## Check Effect of Pruning Rate

In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_IMAGENET_05PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_IMAGENET_025PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.25 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#RestNet18
model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_ResNet18_IMAGENET_01PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
 \nOriginal Model: ResNet18 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)

In [ ]:
#VGG16
model=copy.deepcopy(VGG16_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.5
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_IMAGENET_05PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
 \nOriginal Model: VGG16 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#VGG16
model=copy.deepcopy(VGG16_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.25
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_IMAGENET_025PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
 \nOriginal Model: VGG16 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)


In [ ]:
#VGG16
model=copy.deepcopy(VGG16_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01

SaveTrainStepsName="Progress/PruningIterations_VGG16_IMAGENET_01PR_10Epochs"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.5 and 10 epoch of retraining\
 \nOriginal Model: VGG16 Trained with TINY-IMAGENET Dataset"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe, NumEpochsEachPruneRetrain,MaximumPruneLevel)
